# 用完全連接層估算簡單線性回歸參數

In [4]:
import numpy as np
import torch

In [5]:
# 隨機資料

n = 100
X = np.linspace(0,50,n)
y = np.linspace(0,50,n)

#噪音

X += np.linspace(-10,10,n)
y += np.linspace(-10,10,n)



In [8]:
# 定義模型
# sequential為一系列的層
def create_model(input_feature, output_feature):
    model = torch.nn.Sequential(
        torch.nn.Linear(input_feature, output_feature),
        torch.nn.Flatten(0, -1) # 所有維度轉成一維 
    )
    return model



In [7]:
# 測試扁平層(Flatten)
input = torch.randn(32, 1, 5, 5)
m = torch.nn.Sequential(
    torch.nn.Conv2d(1, 32, 5, 1, 1),
    torch.nn.Flatten()
)
output = m(input)
output.size()

torch.Size([32, 288])

In [9]:
def train(X, y, epochs=2000, lr=1e-6):
    model = create_model(1, 1)

    # 定義損失函數
    loss_fn = torch.nn.MSELoss(reduction='sum') 

    loss_list, w_list, b_list=[], [], []    
    for epoch in range(epochs):   # 執行訓練週期
        y_pred = model(X)        # 預測值
        
        # 計算損失函數值
        # print(y_pred.shape, y.shape)
        MSE = loss_fn(y_pred, y) 
        
        # 梯度重置：改由model.zero_grad() 取代 w、b 逐一設定。
        model.zero_grad()
        
        # 反向傳導
        MSE.backward()  
        
        # 權重更新：改用 model.parameters 取代 w、b 逐一更新 
        with torch.no_grad():
            for param in model.parameters():
                param -= lr * param.grad
        
        # 記錄訓練結果
        linear_layer = model[0]
        if (epoch+1) % 1000 == 0 or epochs < 1000:
            w_list.append(linear_layer.weight[:, 0].item())  # w.item()：轉成常數
            b_list.append(linear_layer.bias.item())
            loss_list.append(MSE.item())
        
    return w_list, b_list, loss_list

In [10]:
# 執行訓練
X2, y2 = torch.FloatTensor(X.reshape(X.shape[0], 1)), torch.FloatTensor(y)
w_list, b_list, loss_list = train(X2, y2)

# 取得 w、b 的最佳解
print(f'w={w_list[-1]}, b={b_list[-1]}')

w=1.0023952722549438, b=-0.09975732117891312


In [16]:
# 使用不同學習率及更多的執行週期訓練
X2, y2 = torch.FloatTensor(X.reshape(X.shape[0], 1)), torch.FloatTensor(y)
w_list, b_list, loss_list = train(X2, y2, epochs=10**5, lr=1e-5)
# 取得 w、b 的最佳解
print(f'w={w_list[-1]}, b={b_list[-1]}')


w=nan, b=nan


100000